In [3]:
import sys
sys.path.append("../../code")

from make_droso_figures import *

In [25]:

import scipy 
from scipy import ndimage

def get_vals(cool, l1, l2):
    vals = np.asarray(cool.matrix(balance=False).fetch(l1, l2))
    vals[np.isnan(vals)] = 0
    v = vals.sum()
    return v


def call(loops_bedtool):
    values, shifted_values, not_shifted, is_shifted, places, shifted_places = {}, {}, [], [], [], []
    for cool_name in cools:
        values[cool_name] = []
        shifted_values[cool_name] = []
    l1, l2 = loops_bedtool[:3], loops_bedtool[3:6]
    newl1 = (l1[0], int(l1[1])-res*wsz, int(l1[2])+res*(wsz))
    newl2 = (l2[0], int(l2[1])-res*wsz, int(l2[2])+res*(wsz))
    c = 0
    for cool_name in cools:
        cool = cools[cool_name]
        try:
            val = get_vals(cool, newl1, newl2)
        except Exception as e:
            print(e)
            return None
            continue
        values[cool_name].append(val)
        if c == 0: # First run around
            place = "_".join([str(x) for x in newl1]) + "_" + "_".join([str(x) for x in newl2])
            places.append(place)
            not_shifted.append(0)
        for shift in range(50, 150, 10):
            try:
                shifted_l1 = (l1[0], int(l1[1])-shift*res-res*wsz, int(l1[2])-shift*res+res*(wsz))
                shifted_l2 = (l2[0], int(l2[1])-shift*res-res*wsz, int(l2[2])-shift*res+res*(wsz))
                shifted_val = get_vals(cool, shifted_l1, shifted_l2)
            except:
                shifted_l1 = (l1[0], int(l1[1])+shift*res-res*wsz, int(l1[2])+shift*res+res*(wsz))
                shifted_l2 = (l2[0], int(l2[1])+shift*res-res*wsz, int(l2[2])+shift*res+res*(wsz))
                shifted_val = get_vals(cool, shifted_l1, shifted_l2)
            shifted_values[cool_name].append(shifted_val)
            if c == 0: # First run around
                shifted_place = "_".join([str(x) for x in shifted_l1]) + "_" + "_".join([str(x) for x in shifted_l2])
                shifted_places.append(shifted_place)
                is_shifted.append(1)
        c += 1
    # print("PLACES ARE", places)
    return values, shifted_values, not_shifted, is_shifted, places, shifted_places 

def rownorm(df):
    df = (df.T/df.mean(axis=1)).T
    return df

def colnorm(df):
    df = (df/df.mean(axis=0))
    return df

In [ ]:
cool_pref = '../../cools/'
exp_pref = '../../obs_exp/expected/replicates'
pbt_file = '../loops/loose/full_loops.loops' 


### NC1-8, NC14, S10-12

In [18]:
import cooler 
import numpy as np
import pandas as pd
import pybedtools as pbt

values = {}
shifted_values = {}
places = []
shifted_places = []


not_shifted = []
is_shifted = []

cools = {}


files = [
        'nc1-8_1_2.base_resolution=100.mcool',
        'nc1-8_2_1.base_resolution=100.mcool',
        'nc1-8_2_2.base_resolution=100.mcool',
        'nc1-8_1_1.base_resolution=100.mcool',
        's10-12_1_1.base_resolution=100.mcool',
        's10-12_1_2.base_resolution=100.mcool',
        's10-12_2_1.base_resolution=100.mcool',
        's10-12_2_2.base_resolution=100.mcool',
        'nc14_1_1.base_resolution=100.mcool',
        'nc14_1_2.base_resolution=100.mcool',
        'nc14_2_1.base_resolution=100.mcool',
        'nc14_2_2.base_resolution=100.mcool',
        'nc14_FED_1_1.base_resolution=100.mcool',
        'nc14_FED_1_2.base_resolution=100.mcool',
        'nc14_FED_rerun_1_1.base_resolution=100.mcool',
        'nc14_FED_rerun_1_2.base_resolution=100.mcool',
        'nc12_mitotic_FED_1_1.base_resolution=100.mcool',
        'nc12_mitotic_FED_1_2.base_resolution=100.mcool' ,   
        'nc12_mitotic_FED_rerun_1_1.base_resolution=100.mcool',
        'nc12_mitotic_FED_rerun_1_2.base_resolution=100.mcool',
        'nc14_FED_1_1.base_resolution=100.mcool',
        'nc14_FED_1_2.base_resolution=100.mcool',
]

import pandas as pd
res = 800

balanced_expected = {}
count_expected = {}
for file in files:
    replicate = file.split(".")[0]
    full_path = cool_pref + file + f"::/resolutions/{res}"
    cools[replicate] = cooler.Cooler(full_path)
    expected_df = pd.read_csv(f'{exp_pref}/expected_{file}_{res}.tsv', sep='\t')
    count_expected[replicate] = {} 
    balanced_expected[replicate] = {} 

    for chrom in cools[replicate].chromsizes.index:
        sub_df = expected_df[expected_df.region == chrom]
        sub_df = sub_df.sort_values('diag')
        diags = sub_df.diag
        balanced_values = sub_df['balanced.avg']
        count_values = sub_df['count.avg']
        balanced_expected[replicate][chrom] = dict(zip(diags, balanced_values))
        count_expected[replicate][chrom] = dict(zip(diags, count_values))
    print("Done with", file)
print("Past Sorting")
for cool_name in cools:
    values[cool_name] = []
    shifted_values[cool_name] = []



In [26]:
from concurrent.futures import ProcessPoolExecutor
n_workers = 30

wsz = 0
res = 800

print(f"Using bedtool file: {pbt_file}")
peaks_loops = list(pbt.BedTool(pbt_file))
with ProcessPoolExecutor(max_workers=n_workers) as e:
    loop_parts = e.map(call, peaks_loops, chunksize=1)

reses = []
for part in loop_parts:
    reses.append(part)

Using bedtool file: ./loops/loose/full_loops.loops


In [27]:
all_values, all_shifted_values, all_not_shifted, all_is_shifted, all_places, all_shifted_places = {}, {}, [], [] ,[] ,[]
for cool_name in cools:
    all_values[cool_name] = []
    all_shifted_values[cool_name] = []

In [28]:
c=0
for part in reses:
    if part is None:
        continue
    else:
        values, shifted_values, not_shifted, is_shifted, places, shifted_places = part
    for cool_name in cools:
        all_values[cool_name] += list(values[cool_name])
        all_shifted_values[cool_name] += list(shifted_values[cool_name])
    all_not_shifted += not_shifted
    all_is_shifted += is_shifted
    all_places += places
    all_shifted_places += shifted_places
    c+=1


In [30]:
import pandas as pd
hic_reads = pd.DataFrame()
for cool_name in cools:
    hic_reads[cool_name] = (all_values[cool_name] + all_shifted_values[cool_name])

hic_reads["shifted"] = all_not_shifted + all_is_shifted
hic_reads['places'] = all_places + all_shifted_places

print(len(all_not_shifted))
final_places = hic_reads['places'].values
diags = []
chroms = []
for place in final_places:
    spl = place.split("_")
    chrom = spl[0]
    start = int(spl[1])
    end = int(spl[4])
    diag = (end-start)//res
    diags.append(diag)
    chroms.append(chrom)

1029


In [31]:
raw_expected_df = pd.DataFrame()
balanced_expected_df = pd.DataFrame()

for cool_name in count_expected:
    raw_values = []
    balanced_values = []    
    for i in range(len(diags)):
        chrom = chroms[i]
        diag = diags[i]
    
        raw_diag_dict = count_expected[cool_name][chrom]
        raw_exp = np.nanmean([raw_diag_dict[x] for x in np.arange(diag-1-wsz, diag+1+wsz+1)])        

        bal_diag_dict = balanced_expected[cool_name][chrom]
        bal_exp = np.nanmean([bal_diag_dict[x] for x in np.arange(diag-1-wsz, diag+1+wsz+1)])                
        
        raw_values.append(raw_exp)
        balanced_values.append(bal_exp)

    assert len(diags) == len(final_places)

    raw_expected_df[cool_name] = raw_values
    balanced_expected_df[cool_name] = balanced_values
    print('Done with', cool_name)
raw_expected_df['diag'] = arr(diags)
balanced_expected_df['diag'] = arr(diags)

Done with nc1-8_1_2
Done with nc1-8_2_1
Done with nc1-8_2_2
Done with nc1-8_1_1
Done with s10-12_1_1
Done with s10-12_1_2
Done with s10-12_2_1
Done with s10-12_2_2
Done with nc14_1_1
Done with nc14_1_2
Done with nc14_2_1
Done with nc14_2_2
Done with nc14_FED_1_1
Done with nc14_FED_1_2
Done with nc14_FED_rerun_1_1
Done with nc14_FED_rerun_1_2
Done with nc12_mitotic_FED_1_1
Done with nc12_mitotic_FED_1_2
Done with nc12_mitotic_FED_rerun_1_1
Done with nc12_mitotic_FED_rerun_1_2


In [39]:
raw_expected_df.columns = [x.replace("14_FED", "14.FED") for x in raw_expected_df.columns]
raw_expected_df.columns = [x.replace("14_for", "14.for") for x in raw_expected_df.columns]
raw_expected_df.columns = [x.replace("FED_rerun", "FED.rerun") for x in raw_expected_df.columns]
raw_expected_df.columns = [x.replace("12_mitotic_FED", "12.mitotic.FED") for x in raw_expected_df.columns]
raw_expected_df.columns = [x.replace("12_mitotic_for", "12.mitotic.for") for x in raw_expected_df.columns]

hic_reads.columns = [x.replace("14_FED", "14.FED") for x in hic_reads.columns]
hic_reads.columns = [x.replace("14_for", "14.for") for x in hic_reads.columns]
hic_reads.columns = [x.replace("FED_rerun", "FED.rerun") for x in hic_reads.columns]
hic_reads.columns = [x.replace("12_mitotic_FED", "12.mitotic.FED") for x in hic_reads.columns]
hic_reads.columns = [x.replace("12_mitotic_for", "12.mitotic.for") for x in hic_reads.columns]

In [34]:
hic_reads = hic_reads.iloc[:, ~hic_reads.columns.str.contains("form")]
raw_expected_df = raw_expected_df.iloc[:, ~raw_expected_df.columns.str.contains("form")]

# (hic_reads).drop(hic_reads.columns[hic_reads.columns.str.contains("FED")], axis=1).to_csv('LOOSE_hic_values_close.csv', index=None,)
(hic_reads).to_csv('../DESEQ2/LOOSE_hic_values_close.csv', index=None)


#### Make normalization df

In [40]:
hic_reads = hic_reads.iloc[:, ~hic_reads.columns.str.contains("form")]
raw_expected_df = raw_expected_df.iloc[:, ~raw_expected_df.columns.str.contains("form")]

In [49]:
raw_exps = raw_expected_df.drop(['diag'], axis=1)#.drop(hic_reads.columns[hic_reads.columns.str.contains("FED")], axis=1)
rownorm(1/raw_exps).to_csv('../DESEQ2/LOOSE_raw_normalization_df.csv', index=None)

# rownorm(1/bal_exps).to_csv('REPLICATE_DESEQ/LOOSE_balanced_normalization_df.csv', index=None)
# rownorm(balancing_df/bal_exps).drop(['diag'], axis=1).to_csv('REPLICATE_DESEQ/LOOSE_rebalanced_balanced_normalization_df.csv', index=None)

In [47]:
def rownorm(df):
    df = (df.T/df.mean(axis=1)).T
    return df